# Introduction

Clustering is a very hard problem because there is never truly 'right' answer when labels are unknown.

To complicate matters further there is **[No Free Lunch](https://en.wikipedia.org/wiki/No_free_lunch_in_search_and_optimization)** for clustering algorithms and while one algorithm might fit a certain dataset well, there are no guarantees that it will work on a different data the exact same way. 
Likewise, clustering is *"strongly dependent on contexts, aims and decisions of the researcher"* which adds fire to the argument that there is no such thing as a *"universally optimal method that will just produce natural clusters"* ( see [What Are True Clusters? Henning 2015](https://arxiv.org/abs/1502.02555) ).

Moreover, clustering techniques that generalize well, such as KMeans, assume that data is numerical and sphere-shaped. Having data of mixed types with high dimensionality also presents challenges for the downstream clustering task as classical methods such as Principal Component Analysis PCA for dimensionality reduction do not work when categorical values are included. This leads to a conundrum for the practitioner where specific featurization schemes must be formalized - such as including only numerical values or transforming all to categorical and then using Multiple Correspondence Analysis MCA instead.

The approach outlined here seeks to solve both the difficulty in finding a default clustering algorithm and to circumvent the difficulties represented when data is in a mixed type form. Using a combination of the [Uniform Manifold Approximation and Projection UMAP](https://arxiv.org/abs/1802.03426) and [Hierarchical Density Based Clustering HDBSCAN](https://arxiv.org/abs/1705.07321) we will first take mix type data and then map it into a dense lower dimensional space. From this dense space, we will then hierarchically group into clusters based on the density of points. The final result will provide a simple end to end to solution that can be applied on wide variety of data to find meaningful clusters.


In [ ]:
import warnings

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics, preprocessing

from denseclus import DenseClus

%matplotlib inline


warnings.filterwarnings("ignore")

In [ ]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set(rc={"figure.figsize": (10, 8)})

In [ ]:
SEED = 42
np.random.seed(SEED)  # set the random seed as best we can

data_url = (
    "https://raw.githubusercontent.com/awslabs/aws-customer-churn-pipeline/main/data/churn.txt"
)
df = pd.read_csv(data_url).sample(n=2000, random_state=SEED)

In this example, you will use a synthetic churn dataset for an imaginary telecommunications company with the outcome Churn? flagged as as either True (churned) or False (did not churn). Features include customer details such as plan and usage information. The churn dataset is publicly available and mentioned in the book [Discovering Knowledge in Data by Daniel T. Larose](https://www.amazon.com/dp/0470908742/). It is attributed by the author to the University of California Irvine Repository of Machine Learning Datasets.

In [ ]:
df.drop(["Phone", "Area Code"], axis=1, inplace=True)
df.dtypes

As you can see the data consists of both categorical and numeric features.
Generally, speaking this is problematic for traditional dimension reduction and clustering methods such as [K-Means](https://en.wikipedia.org/wiki/K-means_clustering) as they rely input features to be numeric and assume that the values are shaped spherical in nature.

With DenseClus this is not an issue because we use create UMAP embeddings for both categorical and numerical, combining the embedding space to output them into the densest space possible. Next HDBSCAN is run to group densities into clusters, resulting a groups of mixed-type data. 

All of this is done under the hood and just requires a `fit` call like below.

In [ ]:
clf = DenseClus(
    random_state=SEED,
    cluster_selection_method="leaf",
    umap_combine_method="intersection_union_mapper",
)

clf.fit(df)

As a recap the steps that happened are:

1). Numerical features were taken out and then reduced into a *dense* UMAP embedding

2) Categorical features got extracted and learned into a *dense* separate UMAP embedding

3) The two embeddings were then combined with an intersection over union operation

4) HDBSCAN uses density-based spatial clustering to hierarchical-fashion to extract clusters from the combined space

All of these features are now attached as usable `DenseClus` object. 

## Checking Embedding Results

Verify the embeddings are now densely shaped.

In [ ]:
for i in range(clf.n_components):
    sns.kdeplot(clf.mapper_.embedding_[:, i], shade=True)

In [ ]:
_ = sns.jointplot(x=clf.mapper_.embedding_[:, 0], y=clf.mapper_.embedding_[:, -1], kind="kde")

# Inspection of Cluster Results

The clustering results are extricable from the object.

X groups formed into clusters, with the largest constituting Y% of the data.

In [ ]:
labels = clf.score()

print(labels, "\n")
print(pd.DataFrame(labels).value_counts(normalize=True))

Just like with DBSCAN, labels of -1 are flagged as noise and all lables have an associated noise probability score.

In practice, these can be thrown out or put into an 'other' cluster.

In [ ]:
_ = sns.jointplot(
    x=clf.mapper_.embedding_[:, 0],
    y=clf.mapper_.embedding_[:, 1],
    hue=labels,
    kind="kde",
)

As you can see above, the clusters formed based on the densities represented in the reduced space.

Specifically, 4 groups got identified by the data.

Since HDBSCAN is hierarchical in nature and splits based on a tree.

Instead of using the default method, you used `leaf` which splits into smaller groups along the tree. 

The below plot shows that the default setting would find two clusters total but since we split along the leaf there are four.

In [ ]:
_ = clf.hdbscan_.condensed_tree_.plot(
    select_clusters=True,
    selection_palette=sns.color_palette("deep", np.unique(labels).shape[0]),
)

# Profiling the Clusters

Finally, once clusters are formed, it's common practice to then describe what each one means.

Here, descriptive statistics is actually a very powerful (and efficient) tool to use.

In [ ]:
df["segment"] = clf.score()

numerics = df.select_dtypes(include=[int, float]).drop(["segment"], 1).columns.tolist()

df[numerics + ["segment"]].groupby(["segment"]).median()

Let's ignore group `-1`.

You can see that only first segment `0` has the shortest day minutes and high night calls. These are perhaps customers that prefer calling (and getting called) in the night.

The second segment `1` has the highest day minutes and day charges. These are customer that prefer calling (and getting called) during the day.

This type of profiling is possible with the other segments as well, coming up with a description of what attributes they constitute and how they relate to your objective.

Again, this is just profiling but descriptive statistics are revealing of what patterns are captured.

A similar type of analysis is possible with categorical features, with a reference shown below.

In [ ]:
categorical = df.select_dtypes(include=["object"]).drop(["State"], 1)

In [ ]:
for c in categorical.columns:
    df.groupby(["segment"] + [c]).size().plot(
        kind="bar", color=sns.color_palette("deep", np.unique(labels).shape[0])
    )
    plt.title(c)
    plt.show()